# Prediction metrics

In [1]:
def LHT(op, l, r):
    if op in Locks[l]:
        return service_time[op] + lock_cost[l][PLACE][r]
    else:
        return 0

    
def op_serialization(l, r):
    res = 0
    for op in Ops:
        if modes[MODE][op] == 'X':
            for r_1 in Reps:
                if r_1 != r:
                    res += LHT(op, l, r_1) * freq[WL][op][r_1]
#     print('os', res)
    return res


def rep_serialization(l, r):
    res = 0
    for op in Ops:
        res += LHT(op, l, r) * freq[WL][op][r]
#     print('rp', res)
    return res


def op_parallelism(l, r):
    res = 0
    s = 0
    for r_1 in Reps:
        if r != r_1:
            for op in Ops:
                if modes[MODE][op] == 'S':
                    s += LHT(op, l, r_1) * freq[WL][op][r_1]
        res = max(res, s)
#     print('op', res)
    return res


def nonparallelism(l):
    res = 0
    for r in Reps:
        np = abs(op_serialization(l, r) + rep_serialization(l, r) - op_parallelism(l,r))
        res = max(res, np)
#     print('np', res)
    return res


def coordination_parallelism():
    res = 0
    for l in Locks:
        np = nonparallelism(l)
        res = max(res, np)
    return res


In [2]:
Locks = {'a-b':{'insert', 'read'}}
Reps = ['houston', 'paris', 'singapore']
Ops = ['insert', 'read']
modes = {1:{'insert':'X', 'read':'X'}, 2:{'insert':'X','read':'S'}, 3:{'insert':'S','read':'X'}}
service_time = {'insert':0.5, 'read':0.5}
lock_cost = {'a-b': {'cent':{'houston':115, 'paris':5, 'singapore':155}, 'clust':{'houston':137, 'paris':137, 'singapore':287}}}
             #, 'dist':{'houston':115, 'paris':5, 'singapore':155}} -- measure this
freq = {'workloadeqeq':{'insert':{'houston':167, 'paris':167, 'singapore':166}, 'read':{'houston':167, 'paris':167, 'singapore':166}},
       'workloadeqhot':{'insert':{'houston':0, 'paris':500, 'singapore':0}, 'read':{'houston':0, 'paris':500, 'singapore':0}},
        'workloadeqclust':{'insert':{'houston':250, 'paris':250, 'singapore':0}, 'read':{'houston':250, 'paris':250, 'singapore':0}},
        'workloadhoteq':{'insert':{'houston':333, 'paris':334, 'singapore':333}, 'read':{'houston':0, 'paris':0, 'singapore':0}},
        'workloadhothot':{'insert':{'houston':0, 'paris':1000, 'singapore':0}, 'read':{'houston':0, 'paris':0, 'singapore':0}},
        'workloadhotclust':{'insert':{'houston':500, 'paris':500, 'singapore':0}, 'read':{'houston':0, 'paris':0, 'singapore':0}},
        'workloadF':{'insert':{'houston':0, 'paris':500, 'singapore':0}, 'read':{'houston':167, 'paris':167, 'singapore':166}},
        'workloadG':{'insert':{'houston':0, 'paris':500, 'singapore':0}, 'read':{'houston':250, 'paris':0, 'singapore':250}}
       }

In [3]:
for WL in freq:
    for MODE in [1,2,3]:
        for PLACE in ['cent']:
            print(WL[8:]+' '*(16-len(WL)), '\t', MODE, '\t', PLACE, '\t', coordination_parallelism())

eqeq     	 1 	 cent 	 92040.0
eqeq     	 2 	 cent 	 51626.0
eqeq     	 3 	 cent 	 51626.0
eqhot    	 1 	 cent 	 5500.0
eqhot    	 2 	 cent 	 5500.0
eqhot    	 3 	 cent 	 5500.0
eqclust  	 1 	 cent 	 60500.0
eqclust  	 2 	 cent 	 57750.0
eqclust  	 3 	 cent 	 57750.0
hoteq    	 1 	 cent 	 92080.0
hoteq    	 2 	 cent 	 92080.0
hoteq    	 3 	 cent 	 88406.0
hothot   	 1 	 cent 	 5500.0
hothot   	 2 	 cent 	 5500.0
hothot   	 3 	 cent 	 5500.0
hotclust 	 1 	 cent 	 60500.0
hotclust 	 2 	 cent 	 60500.0
hotclust 	 3 	 cent 	 60500.0
F        	 1 	 cent 	 48770.0
F        	 2 	 cent 	 41433.0
F        	 3 	 cent 	 48770.0
G        	 1 	 cent 	 70500.0
G        	 2 	 cent 	 65000.0
G        	 3 	 cent 	 70500.0


In [4]:
for WL in freq:
    for MODE in [1,2,3]:
        for PLACE in ['cent']:
            for l in Locks:
                for r in Reps:
                    print(WL[8:]+' '*(16-len(WL)), '\t', MODE, '\t', PLACE, '\t', l, '\t', r[:4], '\t', op_serialization(l, r))

eqeq     	 1 	 cent 	 a-b 	 hous 	 53463.0
eqeq     	 1 	 cent 	 a-b 	 pari 	 90203.0
eqeq     	 1 	 cent 	 a-b 	 sing 	 40414.0
eqeq     	 2 	 cent 	 a-b 	 hous 	 26731.5
eqeq     	 2 	 cent 	 a-b 	 pari 	 45101.5
eqeq     	 2 	 cent 	 a-b 	 sing 	 20207.0
eqeq     	 3 	 cent 	 a-b 	 hous 	 26731.5
eqeq     	 3 	 cent 	 a-b 	 pari 	 45101.5
eqeq     	 3 	 cent 	 a-b 	 sing 	 20207.0
eqhot    	 1 	 cent 	 a-b 	 hous 	 5500.0
eqhot    	 1 	 cent 	 a-b 	 pari 	 0.0
eqhot    	 1 	 cent 	 a-b 	 sing 	 5500.0
eqhot    	 2 	 cent 	 a-b 	 hous 	 2750.0
eqhot    	 2 	 cent 	 a-b 	 pari 	 0.0
eqhot    	 2 	 cent 	 a-b 	 sing 	 2750.0
eqhot    	 3 	 cent 	 a-b 	 hous 	 2750.0
eqhot    	 3 	 cent 	 a-b 	 pari 	 0.0
eqhot    	 3 	 cent 	 a-b 	 sing 	 2750.0
eqclust  	 1 	 cent 	 a-b 	 hous 	 2750.0
eqclust  	 1 	 cent 	 a-b 	 pari 	 57750.0
eqclust  	 1 	 cent 	 a-b 	 sing 	 60500.0
eqclust  	 2 	 cent 	 a-b 	 hous 	 1375.0
eqclust  	 2 	 cent 	 a-b 	 pari 	 28875.0
eqclust  	 2 	 cent 	 a-b 	 sin

In [5]:
for WL in freq:
    for MODE in [1,2,3]:
        for PLACE in ['cent']:
            for l in Locks:
                for r in Reps:
                    print(WL[8:]+' '*(16-len(WL)), '\t', MODE, '\t', PLACE, '\t', l, '\t', r[:4], '\t', rep_serialization(l, r))

eqeq     	 1 	 cent 	 a-b 	 hous 	 38577.0
eqeq     	 1 	 cent 	 a-b 	 pari 	 1837.0
eqeq     	 1 	 cent 	 a-b 	 sing 	 51626.0
eqeq     	 2 	 cent 	 a-b 	 hous 	 38577.0
eqeq     	 2 	 cent 	 a-b 	 pari 	 1837.0
eqeq     	 2 	 cent 	 a-b 	 sing 	 51626.0
eqeq     	 3 	 cent 	 a-b 	 hous 	 38577.0
eqeq     	 3 	 cent 	 a-b 	 pari 	 1837.0
eqeq     	 3 	 cent 	 a-b 	 sing 	 51626.0
eqhot    	 1 	 cent 	 a-b 	 hous 	 0.0
eqhot    	 1 	 cent 	 a-b 	 pari 	 5500.0
eqhot    	 1 	 cent 	 a-b 	 sing 	 0.0
eqhot    	 2 	 cent 	 a-b 	 hous 	 0.0
eqhot    	 2 	 cent 	 a-b 	 pari 	 5500.0
eqhot    	 2 	 cent 	 a-b 	 sing 	 0.0
eqhot    	 3 	 cent 	 a-b 	 hous 	 0.0
eqhot    	 3 	 cent 	 a-b 	 pari 	 5500.0
eqhot    	 3 	 cent 	 a-b 	 sing 	 0.0
eqclust  	 1 	 cent 	 a-b 	 hous 	 57750.0
eqclust  	 1 	 cent 	 a-b 	 pari 	 2750.0
eqclust  	 1 	 cent 	 a-b 	 sing 	 0.0
eqclust  	 2 	 cent 	 a-b 	 hous 	 57750.0
eqclust  	 2 	 cent 	 a-b 	 pari 	 2750.0
eqclust  	 2 	 cent 	 a-b 	 sing 	 0.0
eqclust 

In [6]:
for WL in freq:
    for MODE in [1,2,3]:
        for PLACE in ['cent']:
            for l in Locks:
                for r in Reps:
                    print(WL[8:]+' '*(16-len(WL)), '\t', MODE, '\t', PLACE, '\t', l, '\t', r[:4], '\t', op_parallelism(l, r))

eqeq     	 1 	 cent 	 a-b 	 hous 	 0
eqeq     	 1 	 cent 	 a-b 	 pari 	 0
eqeq     	 1 	 cent 	 a-b 	 sing 	 0
eqeq     	 2 	 cent 	 a-b 	 hous 	 26731.5
eqeq     	 2 	 cent 	 a-b 	 pari 	 45101.5
eqeq     	 2 	 cent 	 a-b 	 sing 	 20207.0
eqeq     	 3 	 cent 	 a-b 	 hous 	 26731.5
eqeq     	 3 	 cent 	 a-b 	 pari 	 45101.5
eqeq     	 3 	 cent 	 a-b 	 sing 	 20207.0
eqhot    	 1 	 cent 	 a-b 	 hous 	 0
eqhot    	 1 	 cent 	 a-b 	 pari 	 0
eqhot    	 1 	 cent 	 a-b 	 sing 	 0
eqhot    	 2 	 cent 	 a-b 	 hous 	 2750.0
eqhot    	 2 	 cent 	 a-b 	 pari 	 0
eqhot    	 2 	 cent 	 a-b 	 sing 	 2750.0
eqhot    	 3 	 cent 	 a-b 	 hous 	 2750.0
eqhot    	 3 	 cent 	 a-b 	 pari 	 0
eqhot    	 3 	 cent 	 a-b 	 sing 	 2750.0
eqclust  	 1 	 cent 	 a-b 	 hous 	 0
eqclust  	 1 	 cent 	 a-b 	 pari 	 0
eqclust  	 1 	 cent 	 a-b 	 sing 	 0
eqclust  	 2 	 cent 	 a-b 	 hous 	 1375.0
eqclust  	 2 	 cent 	 a-b 	 pari 	 28875.0
eqclust  	 2 	 cent 	 a-b 	 sing 	 30250.0
eqclust  	 3 	 cent 	 a-b 	 hous 	 1375